<a href="https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/dev/ytvideo_whisper_KMN_BOT_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-whisper 機器狼特別版

這個專案採用了[Whisper](https://github.com/openai/whisper)而不是以前在用的Decipher專案

如果想要用原本的Whisper專案，就在這裡

# 使用方法

1. 請確認你的執行階段有啟用GPU，在 ``執行階段 -> 變更執行階段類型 ``
1. 更改設定
1. 按下 ``執行階段 -> 執行全部`` 或是 (CTRL+F9)
1. 然後連接到Google雲端硬碟
1. 然後等一下你的檔案應該就會在 ``雲端硬碟 -> Whisper -> result``

## 獲得Google雲端硬碟權限
(在 雲端硬碟 -> Whisper)

In [1]:
#@markdown # **下載選項**
#@markdown ---
#@markdown ## **停用Google雲端硬碟**
disable_google_drive = "no" #@param ['yes', 'no']
# @markdown ## **下載方式** ``bashupload / 直線下載``
download_method = "bashupload" #@param ['bashupload', 'direct']
# @markdown 當啟用 ``結束自動刪除執行階段`` 的時候**無法**使用直線下載，會回到上傳到bashupload的方式。請見諒 </p>
# @markdown **警告：直線下載有時候無法在Firefox或類Firefox瀏覽器 (WaterFox , Librewolf)使用**。如果無法使用的話，請**切換到上傳檔案到bashupload**。
if disable_google_drive.lower() == 'no':

  from google.colab import drive
  import os

  path = '/content/gdrive/MyDrive/Whisper/'
  path2 = '/content/gdrive/MyDrive/Whisper/result'
  drive.mount('/content/gdrive')

  if not os.path.exists(path):
    os.mkdir(path)
    os.mkdir(path2)

# Calcuate the Execution Time.
from datetime import datetime
start_time = datetime.now()

Mounted at /content/gdrive


# 安裝程式
openai-whisper yt-dlp ipython pyperclip wget pycurl

In [2]:
! apt-get install ffmpeg
! pip install openai-whisper yt-dlp ipython pyperclip wget pycurl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 測試GPU是否有開啟

如果沒有東西跑出來的話，請去 ``執行階段 -> 變更執行階段類型`` 更改

In [3]:
! nvidia-smi

Mon May 27 16:56:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 設定


In [4]:
import os
#@markdown # **設定 (星號代表基礎設定)**
#@markdown ---
#@markdown ## **上傳自己的音檔 (目前只限 mp3 檔)**
upload_own_audio_file = "no" #@param ['yes', 'no']
#@markdown ## **YouTube 模式**
#@markdown ---
#@markdown ## ***影片連接**
# YouTube Link
YouTube_Video_Link = "https://www.youtube.com/watch?v=CUzY1lUTnec" #@param {type:"string"}
# File Name Change
#@markdown ## ***檔案名稱**
new_filename = "Infurnity2023「ライフワンダーズスペシャルステージイベント」" #@param {type:"string"}
#@markdown ## ***語言 (自動，中文，英文，日文)**
language = "Auto" #@param ["Auto", "Chinese", "English", "Japanese"]
#@markdown ## ***模式 ``生成 / 翻譯``**
task = "transcribe" #@param ['transcribe', 'translate']
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
new_filename2 = os.path.splitext(new_filename)[0] + " Transcript.txt"
# Change the Model of the Whisper
#@markdown # **其他選項**
#@markdown ---
#@markdown ## **生產逐字稿**
Generate_Plain_Document = "yes" #@param ['yes','no']
#@markdown ## **結束自動刪除執行階段**
shutdown_after_complete = "yes" #@param ['yes','no']
#@markdown ## **模型**
model_user = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large-v1','large-v2','large']
#@markdown ## **Cookies驗證**
enable_cookies = "no" #@param ['yes', 'no']

# 如果你有啟動Cookies驗證
請在這裡上傳Cookies檔案

In [5]:
import os
if enable_cookies.lower() == "yes":
  print("It is not recommended to upload your main account's cookies files here, as this is a shared environment.")
  print("Your cookies might get leaked, and I will not take any responsibility if your account gets stolen.")
  print("The recommended approach is to use another Google Account's cookies to run this tool.")
  print("If it is a private video, you can share the video with that Google Account to grant access to it.")
  print("")

  from google.colab import files
  cookies = files.upload()

  # Specify the path where the files are located
  path_cookies = "/content"

  def rename_files(path_cookies):
      for filename in os.listdir(path_cookies):
          file_path = os.path.join(path_cookies, filename)
          if os.path.isfile(file_path) and filename.endswith(".txt"):
              new_path = os.path.join(path_cookies, "cookies.txt")
              os.rename(file_path, new_path)

  rename_files(path_cookies)

# 下載影片/上傳音檔
使用yt-dlp跟ffmpeg

In [6]:
from google.colab import files
import shutil
import yt_dlp

if upload_own_audio_file == "yes":
  if not os.path.isfile('audio.mp3'):
      own_audio = files.upload()
      if not os.path.exists('content'):
        os.mkdir('content')
        for filename in own_audio.keys():
            os.rename(filename, 'audio.mp3')
            os.replace('audio.mp3', 'content/audio.mp3')

else:
    if enable_cookies.lower() == "yes":
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'outtmpl': 'content/audio',
            'cookiefile': 'cookies.txt'
        }
    else:
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'outtmpl': 'content/audio',
        }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([YouTube_Video_Link])

Saving 苹果M4性能分析：尽力了，但芯片工艺快到头了！ [EbDPvcbilCs].mp3 to 苹果M4性能分析：尽力了，但芯片工艺快到头了！ [EbDPvcbilCs].mp3


# 生成字幕檔

In [7]:
import whisper

model = whisper.load_model(model_user)

print("Whisper model loaded.")


from whisper.utils import get_writer

filename = "audio.mp3"
input_directory = "content"
input_file = f"{input_directory}/{filename}"
if language.lower() == 'auto':
  result = model.transcribe(input_file, verbose=True, task=task)
else:
  result = model.transcribe(input_file, verbose=True, task=task, language=language)

if disable_google_drive.lower() == 'no':

  transcription_root = "/content/gdrive/MyDrive/Whisper/result"

else:
  os.mkdir("result")
  transcription_root = "/content/result"

# Setup Options for SRT/TXT file
options = {
    'max_line_width': None,
    'max_line_count': None,
    'highlight_words': False
}

# Save as an SRT file
srt_writer = get_writer("srt", transcription_root,)
srt_writer(result, new_filename, options)

if Generate_Plain_Document.lower() == "yes":

  # Save as TXT file
  srt_writer = get_writer("txt", transcription_root,)
  srt_writer(result, new_filename2, options)
  print("Generated srt and txt file in the Google Drive -> Whisper -> result Folder")

else:

  print("Generated srt file in the Google Drive -> Whisper -> result Folder")

100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 88.1MiB/s]


Whisper model loaded.
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Chinese
[00:00.000 --> 00:03.040] 没想到 M3 才刚刚推出半年
[00:03.040 --> 00:05.000] 这个月 M4 就已经来了
[00:05.000 --> 00:06.080] 而且最离谱的是
[00:06.080 --> 00:08.200] 它居然是首发在 iPad 上
[00:08.200 --> 00:11.400] 我本来以为作为一个仅仅相隔半年推出的新处理器
[00:11.400 --> 00:15.040] 它或许只是 M3 加强 NPU 换上 N3E 工艺的版本
[00:15.040 --> 00:16.920] 实际上经过这几天的测试
[00:16.920 --> 00:18.520] M4 已经向我们证明了
[00:18.520 --> 00:20.360] 它是值得拥有一个新名字的
[00:20.360 --> 00:23.520] 虽然它现在还禁锢在 iPad Pro 的小身板里
[00:23.520 --> 00:27.960] 但这并不影响我们通过各种方法了解它的架构、性能和能效
[00:28.040 --> 00:31.520] 这次测试 M4 我其实主要想了解两件事情
[00:31.520 --> 00:33.360] 一是通过测试它的新架构
[00:33.360 --> 00:36.080] 来看看苹果今年的性能上限如何
[00:36.080 --> 00:39.640] 包括未来的新 Mac 甚至新的 iPhone 有多大的潜力
[00:39.640 --> 00:42.800] 第二是作为全世界首颗 N3E 工艺的芯片
[00:42.800 --> 00:44.480] 它的能耗表现如何
[00:44.480 --> 00:45.520] 因为今年年底
[00:45.520 --> 00:48.920] 各家的手机芯片都会使用 N3E 工艺来制造
[00:48.920 --> 00:53.320] 这也可以从侧面为

# 壓縮檔案並上傳
如果雲端硬碟被停用了

In [8]:
import shutil
import os.path
import requests
import pycurl
from io import BytesIO

if disable_google_drive.lower() == 'yes':
    # Creating the ZIP file
    filenametime = datetime.now().strftime("%m-%d-%Y-%H-%M-%S")
    archived = shutil.make_archive(filenametime, 'zip', '/content/result')

    if os.path.exists(archived):
        print(archived)

        if shutdown_after_complete.lower() == "yes":
          print("Because you can't use direct download with shutdown after complete. So we decided to switch to bashupload insead.")
          file_path = archived
          upload_url = 'bashupload.com'
          new_new_filename = f'{archived}.zip'

          # Initialize a cURL object
          c  = pycurl.Curl()

          # Prepare the form data
          c.setopt(c.URL, upload_url)
          c.setopt(c.HTTPPOST, [
              ('file', (
                # Set the filename in the form data
                  c.FORM_FILE, file_path,
                  c.FORM_FILENAME, new_new_filename
                )),
            ])

          # Capture the response
          response = BytesIO()
          c.setopt(c.WRITEFUNCTION, response.write)

          # Perform the file upload
          c.perform()
          data = response.getvalue().decode("UTF-8")
          print(data)

        elif download_method.lower() == "bashupload":
          file_path = archived
          upload_url = 'bashupload.com'
          new_new_filename = f'{new_filename}.zip'

          # Initialize a cURL object
          c  = pycurl.Curl()

          # Prepare the form data
          c.setopt(c.URL, upload_url)
          c.setopt(c.HTTPPOST, [
              ('file', (
                # Set the filename in the form data
                  c.FORM_FILE, file_path,
                  c.FORM_FILENAME, new_new_filename
                )),
            ])

          # Capture the response
          response = BytesIO()
          c.setopt(c.WRITEFUNCTION, response.write)

          # Perform the file upload
          c.perform()
          data = response.getvalue().decode("UTF-8")
          print(data)

        else:
            files.download(archived)
            print("File downloaded successfully.")

    else:
        print("ZIP file not created")

# 自動關閉執行階段

In [ ]:
import time
end_time = datetime.now()
print('\u751f\u7522\u7e3d\u6642\u9577: {}'.format(end_time - start_time))
time.sleep (2)

if shutdown_after_complete.lower() == "yes":
    print("Shutting Down the Runtime Because Shutdown After Complete is on")
    from google.colab import runtime
    runtime.unassign()
